In [1]:
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
import os
import glob

In [2]:
folder_path = "../../Data/"

file_list = glob.glob(os.path.join(folder_path, "*.parquet"))

df = [pd.read_parquet(file) for file in file_list]
df = pd.concat(df, ignore_index=True)

In [3]:
df, _ = train_test_split(df, train_size=50000, stratify=df['time_to_stop_activity'], random_state=11)

In [4]:
df = df.drop(
    columns = [
        "user", 
        'project', 
        'current_month', 
        'turnover_num',
        'turnover',
        'betweenness_centrality_12_intercept',
        'betweenness_centrality_12_slope', 
        'betweenness_centrality_12_std_dev',
        'betweenness_centrality_3_intercept', 
        'betweenness_centrality_3_slope',
        'betweenness_centrality_3_std_dev',
        'betweenness_centrality_6_intercept', 
        'betweenness_centrality_6_slope',
        'betweenness_centrality_6_std_dev',
        'betweenness_centrality_9_intercept', 
        'betweenness_centrality_9_slope',
        'betweenness_centrality_9_std_dev'
    ], axis = 1
)

In [5]:
df = df.astype(np.float64)
df = df.replace([np.inf], np.nan)

for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

df = df.replace([-np.inf], np.nan)

for column in df.columns:
    max_value = df[column].min(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [6]:
for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [7]:
def train_evaluate_radius_neighbors(radius, weights, metric, X_train, Y_train, X_test, Y_test):
    mse = None
    mae = None
    r2 = None

    try:
        rnr = RadiusNeighborsRegressor(radius=radius, weights=weights, metric=metric)
        rnr.fit(X_train, Y_train)

        y_pred = []
        neighborhoods = rnr.radius_neighbors(X_test, return_distance=False)
        default_value = Y_train.mean()

        for i, neighbors in enumerate(neighborhoods):
            if len(neighbors) == 0: 
                y_pred.append(default_value)
            else: 
                y_pred.append(rnr.predict(X_test[i].reshape(1, -1))[0])

        mse = mean_squared_error(Y_test, y_pred)
        mae = mean_absolute_error(Y_test, y_pred)
        r2 = r2_score(Y_test, y_pred)

        print(f'RNR - Radius: {radius}, Weights: {weights}, Metric: {metric} Finalized - MSE: {mse}, MAE: {mae}, R2: {r2}')

        return {
            'radius': radius,
            'weights': weights,
            'metric': metric,
            'mean_squared_error': mse,
            'mean_absolute_error': mae,
            'r2_score': r2
        }
    
    except Exception as e:
        print(f'RNR - Radius: {radius}, Weights: {weights}, Metric: {metric} Error: {e}')
    
        return {
            'radius': radius,
            'weights': weights,
            'metric': metric,
            'mean_squared_error': mse,
            'mean_absolute_error': mae,
            'r2_score': r2
        }

In [8]:
param_combinations = [
    (radius, weights, metric)
    for radius in [1.0, 2.0, 3.0, 4.0, 5.0, 10.0, 20.0]
    for weights in ['uniform', 'distance']
    for metric in ['euclidean', 'manhattan']
]

In [9]:
x = df.drop(columns = ['time_to_stop_activity'], axis = 1).values
y = df['time_to_stop_activity'].values

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size = 0.3, 
    random_state=11
)

In [11]:
results = Parallel(n_jobs=3)(
    delayed(train_evaluate_radius_neighbors)(radius, weights, metric, X_train, Y_train, X_test, Y_test)
    for radius, weights, metric in param_combinations
)

In [12]:
benchmark_df = pd.DataFrame(results)

In [13]:
benchmark_df.to_excel('RadiusRegressorBenchmark.xlsx')